In [5]:
import cv2
import numpy as np

# --- Load image WITHOUT resizing ---
image_path = "./zone_199.png"
img = cv2.imread(image_path)
if img is None:
    raise ValueError(f"Failed to load image at {image_path}")

# Get original size
height, width = img.shape[:2]
print(f"Loaded image: {width}x{height}")

# Convert to HSV for color-based filtering
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# --- Define HSV thresholds ---
# Red (resistance)
lower_red1 = np.array([0, 50, 50])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 50, 50])
upper_red2 = np.array([180, 255, 255])

# Green (support)
lower_green = np.array([40, 50, 50])
upper_green = np.array([90, 255, 255])

# --- Create masks ---
red_mask = cv2.bitwise_or(
    cv2.inRange(hsv, lower_red1, upper_red1),
    cv2.inRange(hsv, lower_red2, upper_red2)
)
green_mask = cv2.inRange(hsv, lower_green, upper_green)

# --- Initialize multi-class mask SAME SIZE as image ---
zone_mask = np.zeros((height, width), dtype=np.uint8)

# --- Function to fill in detected zones ---
def apply_zones(binary_mask, class_id, min_w=50, min_h=10):
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > min_w and h > min_h:
            cv2.drawContours(zone_mask, [cnt], -1, color=class_id, thickness=-1)

# --- Detect and apply zones ---
apply_zones(red_mask, class_id=1)
apply_zones(green_mask, class_id=2)

# --- Save final mask (unaltered resolution) ---
cv2.imwrite("./zone_multiclass_mask.png", zone_mask * 127)  # Scaled for grayscale visualization
print("✅ Saved perfectly-aligned mask to ./zone_multiclass_mask.png")


Loaded image: 1440x800
✅ Saved perfectly-aligned mask to ./zone_multiclass_mask.png
